In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
class UNET_down(nn.Module):
    def __init__(self):
        super(UNET_down, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv1_ = nn.Conv2d(64, 64, 3, padding=1)
        
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv2_ = nn.Conv2d(128, 128, 3, padding=1)
        
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3_ = nn.Conv2d(256, 256, 3, padding=1)
        
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4_ = nn.Conv2d(512, 512, 3, padding=1)
        
        self.BN1 = nn.BatchNorm2d(64)
        self.BN2 = nn.BatchNorm2d(128)
        self.BN3 = nn.BatchNorm2d(256)
        self.BN4 = nn.BatchNorm2d(512)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2, 2)
        
        # will be more since we will have more layers
        
    def forward(self, x):
        
        #building unet parts (3 operations for each step)
        x1 = self.conv1(x)
        x1 = self.BN1(x1)
        x1 = self.relu(x1)
        
        x1 = self.conv1_(x1)
        x1 = self.BN1(x1)
        x1 = self.relu(x1) #256x256x64
        x2 = self.maxpool(x1) #128x128x64
        
        x2 = self.conv2(x2)
        x2 = self.BN2(x2)
        x2 = self.relu(x2)
        
        x2 = self.conv2_(x2)
        x2 = self.BN2(x2)
        x2 = self.relu(x2) #128x128x128
        x3 = self.maxpool(x2) #64x64x128
        
        x3 = self.conv3(x3)
        x3 = self.BN3(x3)
        x3 = self.relu(x3)
        
        x3 = self.conv3_(x3)
        x3 = self.BN3(x3)
        x3 = self.relu(x3) #64x64x256
        x4 = self.maxpool(x3) #32x32x256
        
        x4 = self.conv4(x4)
        x4 = self.BN4(x4)
        x4 = self.relu(x4)
        
        x4 = self.conv4_(x4)
        x4 = self.BN4(x4)
        x4 = self.relu(x4) #32x32x512
        x5 = self.maxpool(x4) #16x16x512
        
        x5 = self.conv4_(x5)
        x5 = self.BN4(x5)
        x5 = self.relu(x5)
        
        x5 = self.conv4_(x5)
        x5 = self.BN4(x5)
        x5 = self.relu(x5) #16x16x512
        
        return x1, x2, x3, x4, x5

In [ ]:
class UNET_up(nn.Module):
    def __init__(self):
        super(UNET_up, self).__init_()

        self.relu = nn.ReLU()
        
        self.up5 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv5 = nn.Conv2d(1024, 512, 3, padding=1)
        self.conv5_ = nn.Conv2d(512, 256, 3, padding=1)
        
        self.BN5 = nn.BatchNorm2d(512)
        self.BN5_ = nn.BatchNorm2d(256)
        
        #(...)
        
    def forward(x1, x2, x3, x4, x5):
        
        x5 = self.up5(x5)
        x = torch.cat([x4, x5], dim=1) # batch_size, channel, H, W
        
        x = self.conv5(x)
        x = self.BN5(x)
        x = self.relu(x)
        
        x = self.conv5_(x)
        x = self.BN5_(x)
        x = self.relu(x)
        
        
        
        
        
        

In [ ]:
class UNET_up(nn.Module):
    def __init__(self):
        super(UNET_up, self).__init__()

        self.relu = nn.ReLU()

        self.up5 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv5 = nn.Conv2d(1024, 512, 3, padding=1)
        self.conv5_ = nn.Conv2d(512, 256, 3, padding=1)

        self.BN5 = nn.BatchNorm2d(512)
        self.BN5_ = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(512, 256, 3, padding=1)
        self.conv4_ = nn.Conv2d(256, 128, 3, padding=1)

        self.BN4 = nn.BatchNorm2d(256)
        self.BN4_ = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(256, 128, 3, padding=1)
        self.conv3_ = nn.Conv2d(128, 64, 3, padding=1)

        self.BN3 = nn.BatchNorm2d(128)
        self.BN3_ = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(128, 64, 3, padding=1)
        self.conv2_ = nn.Conv2d(64, 64, 3, padding=1)

        self.BN2 = nn.BatchNorm2d(64)
        self.BN2_ = nn.BatchNorm2d(64)

        self.convf = nn.Conv2d(64,1,1,padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x1, x2, x3, x4, x5):
        x5 = self.up5(x5)
        x = torch.cat([x4, x5], dim=1)

        x = self.conv5(x)
        x = self.BN5(x)
        x = self.relu(x)

        x = self.conv5_(x)
        x = self.BN5_(x)
        x = self.relu(x)

        x = self.up5(x)
        x = torch.cat([x3, x], dim=1)

        x = self.conv4(x)
        x = self.BN4(x)
        x = self.relu(x)

        x = self.conv4_(x)
        x = self.BN4_(x)
        x = self.relu(x)

        x = self.up5(x)
        x = torch.cat([x2, x], dim=1)

        x = self.conv3(x)
        x = self.BN3(x)
        x = self.relu(x)

        x = self.conv3_(x)
        x = self.BN3_(x)
        x = self.relu(x)

        x = self.up5(x)
        x = torch.cat([x1, x], dim=1)

        x = self.conv2(x)
        x = self.BN2(x)
        x = self.relu(x)

        x = self.conv2_(x)
        x = self.BN2_(x)
        x = self.relu(x)

        x = self.convf(x)
        x = self.sigmoid(x)

In [ ]:
class UNET(nn.Module):
    def __init__(self):
        super(UNET,self).__init__()

        self.convdown = UNET_down()
        self.convup = UNET_up()

    def forward(self, x):
        x1, x2, x3, x4, x5 = self.convdown(x)
        x = self.convup(x1, x2, x3, x4, x5)

In [12]:
import os
path = "./DATA/"
for i in os.listdir(path):
    seperate(path+i+"/"+os.listdir(path+i)[0], path+i+"/")

In [11]:
from PIL import Image

def seperate(path, save_path):
# Open the TIFF file
    im = Image.open(path)

    # Iterate over the images in the TIFF file and save each image as a separate file
    i = 0
    try:
        while True:
            im.seek(i)
            im.save(f'{save_path}image_{i}.tif')
            i += 1
    except EOFError:
        pass